In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [2]:
spark

### **Zadanie 1**

In [3]:
sc = spark.sparkContext

In [4]:
def f(x):
  return x**2 + 1

x_min = 0
x_max = 2
y_min = f(x_min)
y_max = f(x_max)

In [5]:
import random

data = []

data_length = 1000000

for i in range(data_length):
  data.append((random.uniform(x_min, x_max), random.uniform(y_min, y_max)))


In [6]:
dist_data = sc.parallelize(data)

def is_under_f(d):
  x = d[0]
  y = d[1]
  return f(x) > y

number_under_f = dist_data.map(is_under_f).reduce(lambda a,b: a + b)

(number_under_f / data_length) * (x_max - x_min) * (y_max - y_min) + 2

4.6681919999999995

In [7]:
spark.stop()

### **Zadanie 2**

In [15]:
spark = SparkSession.builder.getOrCreate()
        
sc = spark.sparkContext

In [16]:
import re

communist = sc.textFile("the_communist_manifesto.txt")
bible = sc.textFile("the_king_james_bible.txt")


In [10]:
communist_tokenized = sc.parallelize(communist.map(lambda x: list(filter(lambda t: t != '', x.split(' ')))).reduce(lambda a, b: a + b))
communist_tokenized = communist_tokenized.map(lambda x: re.sub('[^A-Za-z]+', '', x).lower())

In [ ]:
bible_tokenized = sc.parallelize(bible.map(lambda x: list(filter(lambda t: t != '', x.split(' ')))).reduce(lambda a, b: a + b))
bible_tokenized = bible_tokenized.map(lambda x: re.sub('[^A-Za-z]+', '', x).lower())

In [12]:
communist_tokenized.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).takeOrdered(20, lambda x: -x[1])

[('the', 1137),
 ('of', 779),
 ('and', 344),
 ('in', 287),
 ('to', 262),
 ('a', 163),
 ('is', 135),
 ('that', 125),
 ('by', 112),
 ('it', 99),
 ('with', 99),
 ('bourgeois', 95),
 ('all', 94),
 ('class', 91),
 ('bourgeoisie', 91),
 ('as', 83),
 ('they', 81),
 ('for', 79),
 ('its', 78),
 ('their', 75)]

In [13]:
spark.stop()

In [20]:
communist.map(lambda x: 1).reduce(lambda a,b: a + b)

1487

In [21]:
bible.map(lambda x: 1).reduce(lambda a,b: a + b)

99817